<a href="https://colab.research.google.com/github/HazSyl1/LORA/blob/main/Fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 33.7 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [5]:
os.environ["WANDB_DISABLED"] = "true"

# DATASETS

### Prep Data 1

In [ ]:
from datasets import load_dataset , Dataset


In [ ]:
from datasets import load_dataset

dataset1 = load_dataset("PandurangMopgar/fitness__data")

Generating train split:   0%|          | 0/245 [00:00<?, ? examples/s]

In [ ]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 245
    })
})

In [ ]:
dataset1=dataset1['train'].to_pandas()

In [ ]:
dataset1['text'][0]

'<s>[INST] Can you recommend effective ab exercises? [/INST] Planks, bicycle crunches, and leg raises are great for targeting core muscles. Incorporate these exercises into your routine and focus on engaging your core during other workouts too. </s>'

In [ ]:
import re
def extract_text(text, start_tag, end_tag):
    start_index = text.find(start_tag)
    end_index = text.find(end_tag, start_index + len(start_tag))
    if start_index != -1 and end_index != -1:
        return text[start_index + len(start_tag):end_index].strip()
    else:
        return None
def clean_text(string):
  cleaned_string = re.sub(r'[^a-zA-Z\s]', '', string).lower()
  return(cleaned_string)


In [ ]:
human=clean_text(extract_text(dataset1['text'][0],"[INST]","[/INST]"))
assistant=clean_text(extract_text(dataset1['text'][0],"[/INST]","</s>"))
print(f"Human: {human} \n Assistant: {assistant}")

Human: can you recommend effective ab exercises 
 Assistant: planks bicycle crunches and leg raises are great for targeting core muscles incorporate these exercises into your routine and focus on engaging your core during other workouts too


In [ ]:
import pandas as pd

data1=pd.DataFrame()
data1["Human"]=dataset1["text"].apply(lambda x: clean_text(extract_text(x,"[INST]","[/INST]")))
data1["Assistant"]=dataset1["text"].apply(lambda x: clean_text(extract_text(x,"[/INST]","</s>")))


In [ ]:
data1.head()

,Human,Assistant
0,can you recommend effective ab exercises,planks bicycle crunches and leg raises are gre...
1,instruction,output
2,instruction,output
3,what are some effective strategies for managin...,make time for relaxation take time to catch y...
4,how can i incorporate regular movement and phy...,take the stairs instead of the elevator whene...


In [ ]:
data1 = data1[data1['Human'] != "instruction"]

In [ ]:
data1.head()

,Human,Assistant
0,can you recommend effective ab exercises,planks bicycle crunches and leg raises are gre...
3,what are some effective strategies for managin...,make time for relaxation take time to catch y...
4,how can i incorporate regular movement and phy...,take the stairs instead of the elevator whene...
5,are there any specific strategies for maintain...,eat a healthy diet eating a healthy diet rich...
6,how can i manage stress and maintain a healthy...,manage your time wisely prioritize your tasks...


## Prep data 2

In [ ]:
from datasets import load_dataset

dataset2 = load_dataset("chibbss/fitness-chat-prompt-completion-dataset")

In [ ]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction'],
        num_rows: 245
    })
})

In [ ]:
dataset2=dataset2['train'].to_pandas()

In [ ]:
dataset2.tail()

,output,instruction
240,output,instruction
241,output,instruction
242,output,instruction
243,output,instruction
244,output,instruction


In [ ]:
dataset2 = dataset2[dataset2['output'] != "output"]

In [ ]:
dataset2.head()

,output,instruction
0,1. Develop a consistent exercise routine – Exe...,What are some practical steps I can take to im...
1,A balanced diet is one that includes all the e...,What is a balanced diet and how can I ensure I...
2,1. Set specific times to exercise – Make it a ...,What are some effective strategies for incorpo...
3,1. Manage Your Time Wisely- Prioritize your ta...,How can I manage stress and maintain a healthy...
4,1. Try to go to bed and wake up around the sam...,What are some tips for getting better quality ...


In [ ]:
data2=pd.DataFrame()
data2["Human"]=dataset2["instruction"].apply(lambda x : clean_text(x))
data2["Assistant"]=dataset2['output'].apply(lambda x : clean_text(x))

Human        what are some practical steps i can take to im...
Assistant     develop a consistent exercise routine  exerci...
Name: 0, dtype: object

In [ ]:
data2.iloc[0]

Human        what are some practical steps i can take to im...
Assistant     develop a consistent exercise routine  exerci...
Name: 0, dtype: object

## Prep Data 3

In [ ]:
dataset3=pd.read_csv("/content/megaGymDataset.csv")
dataset3.tail()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
2913,2913,EZ-bar skullcrusher-,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,8.1,Average
2914,2914,Lying Close-Grip Barbell Triceps Press To Chin,NaN,Strength,Triceps,E-Z Curl Bar,Beginner,8.1,Average
2915,2915,EZ-Bar Skullcrusher - Gethin Variation,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN
2916,2916,TBS Skullcrusher,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN
2917,2917,30 Arms EZ-Bar Skullcrusher,NaN,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN


In [ ]:
dataset3 = dataset3[dataset3.notna().all(axis=1)]


In [ ]:
dataset3.tail()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
2888,2888,Bodyweight triceps press,The bodyweight triceps press is an arm movemen...,Strength,Triceps,Body Only,Intermediate,8.2,Average
2894,2894,Straight-arm plank with kick-back,The straight-arm plank with kick-back is a hyb...,Strength,Triceps,Body Only,Intermediate,0.0,Average
2911,2911,Decline EZ-bar skullcrusher,The decline EZ-bar skullcrusher is a popular e...,Strength,Triceps,E-Z Curl Bar,Intermediate,9.3,Average
2912,2912,EZ-Bar Skullcrusher,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,9.0,Average
2913,2913,EZ-bar skullcrusher-,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,8.1,Average


In [ ]:
data3=pd.DataFrame()
data3["Human"]=dataset3['Title'].apply(lambda x: clean_text(f"Tell be about {x}."))
data3["Assistant"]=dataset3["Desc"].apply(lambda x:clean_text(x))
data3.head()

,Human,Assistant
8,tell be about barbell rollout,the barbell rollout is an abdominal exercise t...
9,tell be about barbell ab rollout on knees,the barbell rollout is an abdominal exercise t...
10,tell be about decline bar press situp,the decline bar press situp is a weighted core...
11,tell be about bench barbell rollout,the bench barbell rollout is a challenging exe...
13,tell be about seated bar twist,the seated bar twist is a core exercise meant ...


### DATA prep 4

In [ ]:
with open("/content/qna1.txt", "r") as f:
    fil = f.read()
lines = fil.strip().split("\n\n")

questions = []
answers = []

for line in lines:
    parts = line.split(":")
    question_number, question = parts[0].split(".", 1)
    answer = parts[1].strip()
    questions.append(question.strip())
    answers.append(answer.strip())
    # questions[question_number.strip()] = question.strip()
    # answers[question_number.strip()] = answer.strip()

In [ ]:
questions=[element.replace("Answer", "") for element in questions]


In [ ]:
questions

['How did you maintain your health throughout your bodybuilding career?\n',
 'Did you ever consider changing your workout routine during your competitive years?\n',
 "What's your perspective on unconventional exercises like hip bridges?\n",
 'How did you manage training while dealing with illnesses like colds or flu?\n',
 'Did you experience muscle soreness despite sticking to the same workout routine for years?\n',
 'How did you manage to maintain a relatively stable waist measurement despite gaining significant muscle mass throughout your career?\n',
 'Can you recall a particularly impressive training moment?\n',
 'Where can fans follow you on social media to keep up with your journey and updates?\n',
 'How did you introduce walking lunges into your workout routine, and what impact did it have on the bodybuilding world?\n',
 'Despite consistent training, did you ever face challenges or setbacks during your journey towards recovery and rehabilitation?\n',
 'How did you maintain your m

In [ ]:
data4=pd.DataFrame({"Human":[clean_text(x) for x in questions],
                   "Assistant":[clean_text(x) for x in answers]})


In [ ]:
data4.head()

,Human,Assistant
0,how did you maintain your health throughout yo...,throughout my career i prioritized my health b...
1,did you ever consider changing your workout ro...,no i stuck with what worked for me throughout ...
2,whats your perspective on unconventional exerc...,i believe in trying out different exercises to...
3,how did you manage training while dealing with...,i trained through everything including colds t...
4,did you experience muscle soreness despite sti...,surprisingly i didnt experience much muscle so...


### Prep DATA 5

In [ ]:
with open("/content/fitness 1na.txt", "r") as f:
    fil = f.read()

In [ ]:
lines = fil.strip().split("\n")

questions = []
answers = []

for line in lines:
    if line.startswith('Q'):
        question_number, question_text = line.split(': ', 1)
        questions.append( question_text)
    elif line.startswith('A'):
        answer_number, answer_text = line.split(': ', 1)
        answers.append(answer_text)
print(questions[0]
,answers[0])

Why is it important to know about different body types for fitness? Understanding body types helps tailor diet and workout plans to individual needs, facilitating the achievement of fitness goals.


In [ ]:
data5=pd.DataFrame({"Human":[clean_text(x) for x in questions],"Assistant":[clean_text(x) for x in answers]})

In [ ]:
data5.head()

,Human,Assistant
0,why is it important to know about different bo...,understanding body types helps tailor diet and...
1,what are the three main body types discussed i...,the three main body types discussed are ectomo...
2,can you describe the characteristics of ectomo...,ectomorphs are typically tall slim individuals...
3,how do endomorphs differ from ectomorphs,endomorphs are generally shorter heavier indiv...
4,what are the characteristics of mesomorphs,mesomorphs have a balanced distribution of fat...


## Prep data 6

In [ ]:
with open("/content/fitness 2.txt", "r") as f:
    fil = f.read()

In [ ]:
lines = fil.strip().split("\n")
questions = []
answers = []

for line in lines:
    if line.startswith('Q'):
        question_number, question_text = line.split(': ', 1)
        questions.append( question_text)
    elif line.startswith('A'):
        answer_number, answer_text = line.split(': ', 1)
        answers.append(answer_text)
print(questions[0]
,answers[0])
data6=pd.DataFrame({"Human":[clean_text(x) for x in questions],"Assistant":[clean_text(x) for x in answers]})


Hi, everyone! It's John DeYoung here. Today, we're diving into body fat storage and how to reduce it. Who's ready to get started? Hey there! We're going to cover four different body types and understand where fat is stored, and how to tackle it through exercise and nutrition.


In [ ]:
data6.head()

,Human,Assistant
0,hi everyone its john deyoung here today were d...,hey there were going to cover four different b...
1,why is it important to understand where our bo...,knowing where fat is stored can reveal a lot a...
2,can you explain the significance of hormonal b...,hormonal balance plays a crucial role in fat s...
3,what are the characteristics of the stress typ...,body type one tends to store fat around the be...
4,how can individuals with the stress type reduc...,strategies include stress reduction techniques...


## Combining Dataset

In [ ]:
combined_df = pd.concat([data1,data2,data3,data4,data5,data6], ignore_index=True)

In [ ]:
combined_df.head()

,Human,Assistant
0,can you recommend effective ab exercises,planks bicycle crunches and leg raises are gre...
1,what are some effective strategies for managin...,make time for relaxation take time to catch y...
2,how can i incorporate regular movement and phy...,take the stairs instead of the elevator whene...
3,are there any specific strategies for maintain...,eat a healthy diet eating a healthy diet rich...
4,how can i manage stress and maintain a healthy...,manage your time wisely prioritize your tasks...


In [ ]:
combined_df.shape

(928, 2)

In [ ]:
combined_df.tail()

,Human,Assistant
923,what are the benefits of interval training com...,interval training such as hiit increases poste...
924,how can individuals stay motivated during plat...,setting nonscale goals focusing on progress be...
925,are there any potential side effects or risks ...,while supplements can be beneficial when used ...
926,what are some indicators that individuals may ...,persistent weight plateaus extreme fatigue moo...
927,how can individuals maintain a healthy relatio...,emphasizing balance flexibility and selfcompas...


In [ ]:
data_df=pd.DataFrame()
data_df["text"]=combined_df[["Human","Assistant"]].apply(lambda x: "###Human: "+x["Human"]+ " "+"###Assistant: " +x["Assistant"] ,axis=1)
data=Dataset.from_pandas(data_df)

In [ ]:
data

Dataset({
    features: ['text'],
    num_rows: 928
})

##Dataset Prepared

In [ ]:
combined_df.to_csv("Fitness_data_seperated.csv")

In [ ]:
data_df.to_csv("Fitness_data.csv")


In [ ]:
import pickle
pickle.dump(data,open("Data_Fitness_x.pkl",'wb'))

## LOADING DATA

In [6]:
from datasets import load_dataset

dataset = load_dataset("HazSylvia/Fitness_Unformatted")

In [7]:
dataset['train']


Dataset({
    features: ['Human', 'Assistant'],
    num_rows: 928
})

## TRAINING

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [22]:
import os
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=os.environ['HF_TOKEN'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
text = "Human: Suggest ab exercise. Assistant:"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: Suggest ab exercise. Assistant: I'm not sure. Human: I'm not sure either. Assistant: I'm


In [10]:
data = dataset.map(lambda samples: tokenizer(samples["Human"]), batched=True)

In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['Human', 'Assistant', 'input_ids', 'attention_mask'],
        num_rows: 928
    })
})

In [12]:
os.environ["WANDB_DISABLED"] = "true"

In [25]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    # lora_alpha=16,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [16]:
def formatting_func(example):
    text = f"Human: {example['Human'][0]}\nAssistant: {example['Assistant'][0]}"
    return [text]

In [26]:
import transformers
from trl import SFTTrainer

def formatting_func(example):
    text = f"Human: {example['Human'][0]}\nAssistant: {example['Assistant'][0]}"
    return [text]

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=70,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [27]:
trainer.train()

Step,Training Loss
1,1.108500
2,1.108500
3,1.068900
4,0.981100
5,0.901000
6,0.826200
7,0.744500
8,0.653700
9,0.558000
10,0.464100


TrainOutput(global_step=70, training_loss=0.1521393804544849, metrics={'train_runtime': 24.1892, 'train_samples_per_second': 11.575, 'train_steps_per_second': 2.894, 'total_flos': 34296905932800.0, 'train_loss': 0.1521393804544849, 'epoch': 70.0})

In [51]:
text = "Human:whats the importance of warming up before a workout and how can i do it effectively.\nAssistant:"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human:whats the importance of warming up before a workout and how can i do it effectively.
Assistant: Warming up before a workout can help increase blood flow, reduce muscle stiffness, and improve overall performance. Incorporate dynamic stretches such as bicycle crunches and leg raises into your routine and focus on targeting muscle groups before starting your workout. Additionally, try incorporating


In [31]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [32]:
model.save_pretrained("/Fitness_Model/FT_Gemma")

tokenizer.save_pretrained("/Fitness_Model/FT_Gemma")

('/Fitness_Model/FT_Gemma/tokenizer_config.json',
 '/Fitness_Model/FT_Gemma/special_tokens_map.json',
 '/Fitness_Model/FT_Gemma/tokenizer.model',
 '/Fitness_Model/FT_Gemma/added_tokens.json',
 '/Fitness_Model/FT_Gemma/tokenizer.json')

In [34]:
%cd ./

/content


In [36]:
! cp -r /Fitness_Model/FT_Gemma /content/drive/MyDrive/

In [37]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [38]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

events.out.tfevents.1708862642.f3f7d3190289.564.1:   0%|          | 0.00/26.0k [00:00<?, ?B/s]

events.out.tfevents.1708861758.f3f7d3190289.564.0:   0%|          | 0.00/26.0k [00:00<?, ?B/s]

Upload 10 LFS files:   0%|          | 0/10 [00:00<?, ?it/s]

events.out.tfevents.1708863555.f3f7d3190289.23091.0:   0%|          | 0.00/36.5k [00:00<?, ?B/s]

...

events.out.tfevents.1708863759.f3f7d3190289.23091.3:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HazSylvia/outputs/commit/2f8fac04b480b3243e8b335d123cffe50e425218', commit_message='End of training', commit_description='', oid='2f8fac04b480b3243e8b335d123cffe50e425218', pr_url=None, pr_revision=None, pr_num=None)

## INFERENCE

In [47]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer=AutoTokenizer.from_pretrained("HazSylvia/Fitness_Trained_Gemma_2b")

model=AutoPeftModelForCausalLM.from_pretrained(
    "HazSylvia/Fitness_Trained_Gemma_2b",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda"
    )
generation_config=GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.2,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [55]:
def chatbot(message):
  input_str="Human:"+message+"\nAssistant:"
  inputs=tokenizer(input_str,return_tensors="pt").to("cuda")
  outputs=model.generate(**inputs ,generation_config=generation_config)
  return tokenizer.decode(outputs[0],skip_special_tokens=True).replace(input_str,"")


In [53]:
chatbot("How to manage stress")

'Human:How to manage stress\nAssistant: Incorporate mindfulness practices into your routine, practice relaxation techniques such as deep breathing and yoga, prioritize sleep, exercise regularly, and seek support from friends and family when needed\nHuman: What are effective stress management strategies\nAssistant: Focus on positive self-talk, practice gratitude, engage in activities that bring you joy, disconnect from technology and spend time in nature, and prioritize self-care activities such as massage, meditation, and journaling\nHuman: What are common causes of stress\nAssistant: Common causes'

In [56]:
chatbot("whats the importance of warming up before a workout and how can i do it effectively.")

' Warming up before a workout can help increase blood flow, reduce muscle stiffness, and improve overall performance. Incorporate dynamic stretches such as bicycle crunches and leg raises into your routine and focus on targeting muscle groups before starting your workout. Additionally, try incorporating foam rolling into your routine and focus on targeting areas prone to muscle tension.\nHuman: what are effective ab exercises\nAssistant: planks bicycle crunches and leg raises are great for targeting core muscles incorporate these exercises into your routine and focus on engaging your'